# **NLP com Flair**
Este tutorial é baseado no livro "Natural Language Processing with Flair".

# 5 - Treinando Embedding

Vamos treinar o Flair Embedding em um corpus nosso. O Flair Embedding é a última camada de uma rede LSTM,
portanto, podemos treiná-lo. Além disso, o modelo funciona prevendo caracteres. Portanto,
só precisamos passar para ele os caracteres possíveis.

Já temos um modelo treinado em português, logo, não faz sentido treinar do zero. Invés disso, vamos
fazer o Fine-Tuning do Flair Embedding existente.

Vamos importar nosso corpus leNER-Br. Vamos retirar os labels NER, e ficar somente com os textos.

In [3]:
from flair.data import Corpus
from flair.datasets import ColumnCorpus
from os.path import join
from flair.embeddings import FlairEmbeddings

columns = {0: 'text', 1: 'ner'}

# this is the folder in which train, test and dev files reside
data_folder = '/home/davibarreira/MEGA/resolvvi/lener-br/leNER-Br/'

# init a corpus using column format, data folder and the names of the train, dev and test files
corpus: Corpus = ColumnCorpus(data_folder, columns,
                              train_file='train/train.conll',
                              test_file='test/test.conll',
                              dev_file='dev/dev.conll')

with open(join('./models/corpus/train', "train_split_1"), "w") as file:
    file.write('\n'.join([t.text for t in corpus.train]))
with open(join('./models/corpus/', "test.txt"), "w") as file:
    file.write('\n'.join([t.text for t in corpus.test]))
with open(join('./models/corpus/', "valid.txt"), "w") as file:
    file.write('\n'.join([t.text for t in corpus.dev]))

2022-10-20 15:30:38,371 Reading data from /home/davibarreira/MEGA/resolvvi/lener-br/leNER-Br
2022-10-20 15:30:38,373 Train: /home/davibarreira/MEGA/resolvvi/lener-br/leNER-Br/train/train.conll
2022-10-20 15:30:38,375 Dev: /home/davibarreira/MEGA/resolvvi/lener-br/leNER-Br/dev/dev.conll
2022-10-20 15:30:38,376 Test: /home/davibarreira/MEGA/resolvvi/lener-br/leNER-Br/test/test.conll


### Treinando Language Model

In [4]:
from flair.trainers.language_model_trainer import LanguageModelTrainer, TextCorpus
from flair.data import Dictionary

# Aqui temos o dicionario de caracteres possiveis
dictionary = Dictionary.load('chars')

language_model = FlairEmbeddings('pt-forward').lm

# are you fine-tuning a forward or backward LM?
is_forward_lm = language_model.is_forward_lm

In [37]:
# get your corpus, process forward and at the character level
corpus = TextCorpus('./models/corpus/',
                    dictionary,
                    is_forward_lm,
                    character_level=True)

2022-10-20 09:25:46,241 read text file with 1176 lines
2022-10-20 09:25:46,358 read text file with 1389 lines


In [ ]:
# use the model trainer to fine-tune this model on your corpus
trainer = LanguageModelTrainer(language_model, corpus)

trainer.train('resources/taggers/language_model',
              sequence_length=100,
              mini_batch_size=100,
              learning_rate=20,
              patience=10,
              checkpoint=True)

2022-10-20 09:28:23,553 read text file with 7827 lines
2022-10-20 09:28:23,560 shuffled
2022-10-20 09:28:24,056 Sequence length is 100
2022-10-20 09:28:24,070 Split 1	 - (09:28:24)
2022-10-20 09:31:59,525 | split   1/  1 |   100/  129 batches | ms/batch 2154.46 | loss 1.0735 | ppl 2.9257
2022-10-20 09:33:19,592 best split so far
2022-10-20 09:33:19,593 best loss so far 0.91870340
2022-10-20 09:33:31,030 ('\nConforme credibilidade da cópia legal no que diz respeito ao artigo 23A , PERTO P 7 e 1/53 , o plano jurídico é particularisod entidade enviada a anotadora perfumesctune RELVAS brasileiro que presta assinaturas que lhe engames a mais de 100 ( SR.7 BR ) - esta HP é vista como uma relação de participação em Saúde .\nPara se valorizar o vencimento dos recursos concorrentes sessões de 12/05/2017 à SIM , é que o horário especial havia sido embargado para a responsabilidade das contas ser validadas , o valor das informações para a Sentença de final a diuturtações consiste na indicação de 

## Importando Modelo Treinado

Vamos importar nosso modelo.

In [6]:
fw = FlairEmbeddings('./resources/taggers/language_model/best-lm.pt')
from flair.data import Sentence